In [1]:
from bs4 import BeautifulSoup
import pickle
import requests
import pandas as pd

In [2]:
def get_sp500_current_table():
    url = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = BeautifulSoup(url.text, 'lxml')

    sp500_table = soup.find('table', {'id': 'constituents'})
    if not sp500_table:
        raise ValueError("The 'constituents' table could not be found.")

    rows = sp500_table.find_all('tr')

    tickers = []
    company_names = []
    industries = []
    sub_industries = []
    headquarters_list = []  
    dates_added = []
    ciks = []

    for row in rows[1:]:
        cols = row.find_all('td')
        if len(cols) < 7:  
            print("Skipping a row due to insufficient columns.")
            continue

        ticker = cols[0].text.strip()
        company_name = cols[1].text.strip()
        industry = cols[3].text.strip()
        sub_industry = cols[4].text.strip()
        headquarters_location = cols[5].text.strip()  
        date_added = cols[6].text.strip()
        cik = cols[7].text.strip() if len(cols) > 7 else None

        tickers.append(ticker)
        company_names.append(company_name)
        industries.append(industry)
        sub_industries.append(sub_industry)
        headquarters_list.append(headquarters_location)  
        dates_added.append(date_added)
        ciks.append(cik)

    df = pd.DataFrame({
        'Ticker': tickers,
        'Company Name': company_names,
        'Industry': industries,
        'Sub-Industry': sub_industries,
        'Headquarters': headquarters_list,  
        'Date Added': dates_added,
        'CIK': ciks
    })

    return df

In [3]:
sp500_df = get_sp500_current_table()

In [5]:
df = pd.DataFrame(sp500_df)
df = df.drop(columns=['Date Added'])
df = df.rename(columns={'Headquarters': 'Date Added'})
df = df.rename(columns={'CIK': 'Founded'})
df

,Ticker,Company Name,Industry,Sub-Industry,Date Added,Founded
0,MMM,3M,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,1902
1,AOS,A. O. Smith,Building Products,"Milwaukee, Wisconsin",2017-07-26,1916
2,ABT,Abbott Laboratories,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1888
3,ABBV,AbbVie,Biotechnology,"North Chicago, Illinois",2012-12-31,2013 (1888)
4,ACN,Accenture,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1989
...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,2011
499,YUM,Yum! Brands,Restaurants,"Louisville, Kentucky",1997-10-06,1997
500,ZBRA,Zebra Technologies,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,1969
501,ZBH,Zimmer Biomet,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1927


In [7]:
df.to_csv('SP500StockInformation.csv')